### **Data Preprocessing**



In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn import metrics

import os

In [ ]:
# Useful Constants

# Those are separate normalised input features for the neural network
INPUT_SIGNAL_TYPES = [
    "x_accel",
    "y_accel",
    "z_accel",
]

# Output classes to learn how to classify
LABELS = [
    "Walking", 
    "Jogging", 
    "Uptairs", 
    "Downstairs", 
    "Sitting", 
    "Standing"
] 

In [ ]:
from numpy import genfromtxt
import pandas as pd
x = genfromtxt('/content/drive/MyDrive/Colab Notebooks/WISDM_x.csv', delimiter=',')
y = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/WISDM_y.csv').to_numpy()

In [ ]:
def create_series(x,y,timestep,overlap):

  slide_step = int(timestep*(1-overlap))
  data_num = int((len(x)/slide_step)-1)
  
  dataset = np.ndarray(shape=(data_num,timestep,len(x[0])))
  labels = list()

  for i in range(data_num):
    labels.append(y[slide_step*(i+1)-1])
    for j in range(timestep):
      dataset[i,j,:] = x[slide_step*i+j,:]

  return dataset,np.array(labels)


In [ ]:
y.shape

(1048574, 1)

In [ ]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
le = LabelEncoder()
y = le.fit_transform(y).reshape(-1,1)


ohe = OneHotEncoder()
y = ohe.fit_transform(y).toarray()


/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# y_pred.append("5")
dataset,labels = create_series(x,y,64,0.5)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset,labels,test_size=0.3, random_state=412)
# X_test, X_val, y_test, y_val = train_test_split(X_toSplit,y_toSplit,test_size=0.25, random_state=412)

In [ ]:
X_train.shape
# y_train[100]

(22936, 64, 3)

In [ ]:
# Input Data 

training_data_count = len(X_train)  # 96092 training series (with 50% overlap between each serie)
test_data_count = len(X_test)  # 41183 testing series
n_steps = len(X_train[0])  # 16 timesteps per series
n_input = len(X_train[0][0])  # 3 input parameters per timestep


# LSTM Neural Network's internal structure
# n_hidden = 32 # Hidden layer num of features
n_classes = 6 # Total classes (should go up, or should go down)


# Training 

learning_rate = 0.0025
lambda_loss_amount = 0.0015
training_iters = training_data_count * 300  # Loop 300 times on the dataset
batch_size = 64
display_iter = 30000  # To show test set accuracy during training


# Some debugging info

print("Some useful info to get an insight on dataset's shape and normalisation:")
print("(X shape, y shape, every X's mean, every X's standard deviation)")
print(X_test.shape, y_test.shape, np.mean(X_test), np.std(X_test))
print("The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.")

Some useful info to get an insight on dataset's shape and normalisation:
(X shape, y shape, every X's mean, every X's standard deviation)
(39321, 16, 3) (39321, 6) 2.873419266367973 6.908464983446615
The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.


### **Model**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
 
def getPositionEncoding(seq_len, d, n=10000):
    P = np.zeros((seq_len, d))
    for k in range(seq_len):
        for i in np.arange(int(d/2)):
            denominator = np.power(n, 2*i/d)
            P[k, 2*i] = np.sin(k/denominator)
            P[k, 2*i+1] = np.cos(k/denominator)
    return P


In [ ]:
pe = getPositionEncoding(seq_len=16,d=3)
c = X_train
for i in range(X_train.shape[0]):
  X_train[i] = X_train[i] + pe

# print(c)
# print('done')
# print(X_train)


In [ ]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs

    #conv1D
    x = layers.Conv1D(256, 3,input_shape=input_shape)(x)
    # print(x.shape)
    x = layers.Conv1D(128, 3)(x)
    # print(x.shape)
    x = layers.Conv1D(64, 3)(x)
    # print(x.shape)
    x = layers.Conv1D(32, 3)(x)
    # print(x.shape)

    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)

In [ ]:
input_shape = X_train.shape[1:]


model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.2,
)

model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
# model.summary()

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

model.fit(X_train,y_train,validation_split=0.1,epochs=200,batch_size=64, callbacks=[callback])

model.evaluate(X_test, y_test, verbose=1)


Epoch 1/200
1291/1291 [==============================] - 27s 18ms/step - loss: 0.7371 - accuracy: 0.7362 - val_loss: 0.5482 - val_accuracy: 0.8014
Epoch 2/200
1291/1291 [==============================] - 22s 17ms/step - loss: 0.5388 - accuracy: 0.8051 - val_loss: 0.4470 - val_accuracy: 0.8265
Epoch 3/200
1291/1291 [==============================] - 22s 17ms/step - loss: 0.4831 - accuracy: 0.8213 - val_loss: 0.4387 - val_accuracy: 0.8288
Epoch 4/200
1291/1291 [==============================] - 22s 17ms/step - loss: 0.4455 - accuracy: 0.8332 - val_loss: 0.4429 - val_accuracy: 0.8335
Epoch 5/200
1291/1291 [==============================] - 23s 18ms/step - loss: 0.4231 - accuracy: 0.8403 - val_loss: 0.4033 - val_accuracy: 0.8379
Epoch 6/200
1291/1291 [==============================] - 22s 17ms/step - loss: 0.4059 - accuracy: 0.8453 - val_loss: 0.3782 - val_accuracy: 0.8449
Epoch 7/200
1291/1291 [==============================] - 23s 18ms/step - loss: 0.3936 - accuracy: 0.8492 - val_loss: 0

[0.27438491582870483, 0.9125149250030518]

In [ ]:
model.evaluate(X_test, y_test)

1229/1229 [==============================] - 7s 6ms/step - loss: 0.2744 - accuracy: 0.9125


[0.27438491582870483, 0.9125149250030518]